In [1]:
import pickle
import chess
import pandas as pd
import numpy as np 
from chess_minimax import get_minimax_move
from chess_random import get_random_move

ModuleNotFoundError: No module named 'minimax_white2'

In [12]:
with open('2014_08_over_2400_chess_data.pickle','rb') as read_file:
    chess_df = pickle.load(read_file)

chess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, Game47674 to Game993220
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BlackElo       106 non-null    object
 1   WhiteElo       106 non-null    object
 2   moves          106 non-null    object
 3   move_num       106 non-null    object
 4   winner         106 non-null    object
 5   Black_Elo_Num  106 non-null    int64 
 6   White_Elo_Num  106 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 6.6+ KB


In [13]:
chess_df
moves = chess_df['moves']

In [14]:
char_list = []
for i in range(1,500):
    char_list.append(str(i) + '.')

move_list = []

for game in moves:
    index = list(moves).index(game)
    allmoves = game.split()
    all_moves = [elem for elem in allmoves if elem not in char_list]
    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        fen = board.fen()
        row = [index, i, fen, all_moves[i]]
        move_list.append(row)

In [15]:
move_df = pd.DataFrame(move_list, columns = ['Game_Num', 'Move_Num', 'FEN', 'Move'])
move_df['Next_Move'] = move_df.Move.shift(-1)
ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move', 'Next_Move'].apply(lambda x : x[:-1])

<ipython-input-15-508f604a7da8>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move', 'Next_Move'].apply(lambda x : x[:-1])


In [21]:
ddf['Rand_Move'] =ddf['FEN'].apply(get_random_move)
ddf['Minimax_Move'] =ddf['FEN'].apply(get_minimax_move)

ddf.head()

Move_Num                                                FEN Move  \
Game_Num                                                                       
0        0         0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...   e4   
         1         1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...   e6   
         2         2  rnbqkbnr/pppp1ppp/4p3/8/4P3/2N5/PPPP1PPP/R1BQK...  Nc3   
         3         3  rnbqkb1r/pppp1ppp/4pn2/8/4P3/2N5/PPPP1PPP/R1BQ...  Nf6   
         4         4  rnbqkb1r/pppp1ppp/4pn2/4P3/8/2N5/PPPP1PPP/R1BQ...   e5   

           Next_Move Rand_Move Minimax_Move  Random_Compare  Minimax_Compare  
Game_Num                                                                      
0        0        e6        c6          Nh6           False            False  
         1       Nc3        d4          Nh3           False            False  
         2       Nf6       Nf6          Ne7           False            False  
         3        e5       Bb5          Nd5           False            False  
         4       Nd5       Ne4          Rg8           False            False

In [22]:
ddf['Random_Compare'] = np.where(ddf['Next_Move'] == ddf['Rand_Move'], True, False)
ddf['Minimax_Compare'] = np.where(ddf['Next_Move'] == ddf['Minimax_Move'], True, False)

In [23]:
ddf['Random_Compare'].value_counts()

False    8451
True      618
Name: Random_Compare, dtype: int64

In [19]:
ddf['Minimax_Compare'].value_counts()

False    8521
True      548
Name: Minimax_Compare, dtype: int64

In [ ]:
ddf['Total_Moves'] = ddf.groupby(['Game_Num'])['Move_Num'].transform(max) + 1
ddf['Percent_Through_Game'] = (ddf['Move_Num']+1)/ddf['Total_Moves']
ddf['Stage_Of_Game'] = np.where(ddf['Percent_Through_Game'] < .3334, 'Beginning',
                       np.where(ddf['Percent_Through_Game'] < .6667, 'Middle',         
                               'End'))
ddf.head()

In [ ]:
pd.crosstab(ddf['Minimax_Compare'], ddf['Stage_Of_Game'])

In [ ]:
pd.crosstab(ddf['Random_Compare'], ddf['Stage_Of_Game'])

In [ ]:
#ddf['NeuralNet_Compare'].value_counts()

In [ ]:
ddf['Move_Num'].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(move_df['Move_Num'])

In [ ]:
test = chess_df[chess_df['move_num'] != 0]

In [ ]:
test.info()

In [ ]:
chess_df['Moves'] = chess_df['move_num'].astype(int)

In [ ]:
chess_df['Moves'].describe()